In [1]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import hvplot.polars
import pandas as pd
import altair as alt
import math

from shared.utils import formalize_data_with_str
from shared.random_rules import RandomRules

In [2]:
rr = RandomRules(
  TOTAL_NUMBERS = 35,
  CHOOSE = 5,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5],
  MAX_B_VALUES = [31, 32, 33, 34, 35]
)

In [3]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=164'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')

In [4]:
lst = []
post_str_b1 = None
post_str_blast = None
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=cols[2])
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_blast = str_combo[-1]

with open('./gn_cahe_v3.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
    'post_str_b1', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 5),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))

TOTAL_LINES=7729


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""13/04/2025""","""4185""","""26 06 13 09 27""","""0609132627""","[6, 9, … 27]",6,9,13,26,27,"""06""","""09""","""13""","""26""","""27""",null,null,false,0,"""03_04_13_01""",-29
"""12/04/2025""","""4184""","""23 12 10 16 03""","""0310121623""","[3, 10, … 23]",3,10,12,16,23,"""03""","""10""","""12""","""16""","""23""","""06""","""27""",false,0,"""07_02_04_07""",-6
"""11/04/2025""","""4183""","""09 10 08 01 12""","""0108091012""","[1, 8, … 12]",1,8,9,10,12,"""01""","""08""","""09""","""10""","""12""","""03""","""23""",true,24,"""07_01_01_02""",-5
"""10/04/2025""","""4182""","""30 33 34 10 21""","""1021303334""","[10, 21, … 34]",10,21,30,33,34,"""10""","""21""","""30""","""33""","""34""","""01""","""12""",false,0,"""11_09_03_01""",8
"""09/04/2025""","""4181""","""29 21 20 28 04""","""0420212829""","[4, 20, … 29]",4,20,21,28,29,"""04""","""20""","""21""","""28""","""29""","""10""","""34""",true,16,"""16_01_07_01""",-33
"""08/04/2025""","""4180""","""10 20 30 05 16""","""0510162030""","[5, 10, … 30]",5,10,16,20,30,"""05""","""10""","""16""","""20""","""30""","""04""","""29""",false,0,"""05_06_04_10""",11
"""07/04/2025""","""4179""","""08 21 22 24 07""","""0708212224""","[7, 8, … 24]",7,8,21,22,24,"""07""","""08""","""21""","""22""","""24""","""05""","""30""",true,16,"""01_13_01_02""",37
"""06/04/2025""","""4178""","""07 35 24 34 14""","""0714243435""","[7, 14, … 35]",7,14,24,34,35,"""07""","""14""","""24""","""34""","""35""","""07""","""24""",true,8,"""07_10_10_01""",-6
"""05/04/2025""","""4177""","""34 33 15 05 16""","""0515163334""","[5, 15, … 34]",5,15,16,33,34,"""05""","""15""","""16""","""33""","""34""","""07""","""35""",true,16,"""10_01_17_01""",-57


In [5]:
import holoviews as hv
def plot_post_col (col: str, post_col: str, test_value: str):
  df_viz = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      post_col
    ).agg(pl.count(post_col).alias('count')) \
    .sort("count", descending=True)
  
  
  df_10_last = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .head(10)
  
  # Agrega una columna que sea el porcentaje de la frecuencia relativa
  total = df_viz['count'].sum()
  df_viz = df_viz.with_columns(
    ( (pl.col('count') / total) * 100 ).alias('percent')
  )
  mean_percent = df_viz['percent'].mean()
  mean_percent_75 = mean_percent * 75 / 50
  mean_percent_25 = mean_percent * 25 / 50

  chart = df_viz.hvplot.bar(
    x=post_col,
    y='percent'
  )

  chart = chart * hv.HLine(mean_percent).opts(color='red')
  chart = chart * hv.HLine(mean_percent_75).opts(color='blue')
  chart = chart * hv.HLine(mean_percent_25).opts(color='blue')
  
  chart.opts(default_tools=["pan"])

  df_jumps_map = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'jumps_map'
    ).agg(pl.count('jumps_map').alias('count')) \
    .sort("count", descending=True)
  
  df_unijump = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'unijump'
    ).agg(pl.count('unijump').alias('count')) \
    .sort("count", descending=True)


  print(f'df_viz FOR {col} = {test_value}')
  display(df_viz)

  display(chart)
  
  print(f'df_10_last FOR {col} = {test_value}')
  display(df_10_last)
  
  print(f'df_jumps_map FOR {col} = {test_value}')
  display(df_jumps_map)
  # display(df_unijump)

  print(f'df_unijump FOR {col} = {test_value}')
  display(df_unijump)

  lst_post_str_b = df_viz.filter(pl.col('percent') >= mean_percent)[post_col].to_list()
  lst_jumps_map = df_jumps_map['jumps_map'].to_list() # Used to discard
  
  return lst_post_str_b, lst_jumps_map

In [6]:
possible_b1, exclude_jump_maps_b1 = plot_post_col(col='str_b1', post_col='post_str_b1', test_value="06")

df_viz FOR str_b1 = 06


post_str_b1,count,percent
str,u32,f64
"""01""",81,14.136126
"""02""",67,11.692845
"""04""",61,10.645724
"""06""",49,8.551483
"""05""",49,8.551483
…,…,…
"""18""",3,0.52356
"""21""",2,0.34904
"""23""",1,0.17452


:Overlay
   .Bars.I    :Bars   [post_str_b1]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b1 = 06


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""26/03/2025""","""4167""","""34 29 06 24 26""","""0624262934""","[6, 24, … 34]",6,24,26,29,34,"""06""","""24""","""26""","""29""","""34""","""05""","""35""",true,16,"""18_02_03_05""",-16
"""24/03/2025""","""4165""","""32 29 27 21 06""","""0621272932""","[6, 21, … 32]",6,21,27,29,32,"""06""","""21""","""27""","""29""","""32""","""04""","""35""",false,0,"""15_06_02_03""",0
"""10/03/2025""","""4151""","""29 06 26 15 19""","""0615192629""","[6, 15, … 29]",6,15,19,26,29,"""06""","""15""","""19""","""26""","""29""","""02""","""32""",true,16,"""09_04_07_03""",-15
"""05/03/2025""","""4146""","""22 18 16 06 20""","""0616182022""","[6, 16, … 22]",6,16,18,20,22,"""06""","""16""","""18""","""20""","""22""","""13""","""35""",true,28,"""10_02_02_02""",-8
"""06/02/2025""","""4119""","""11 14 25 20 06""","""0611142025""","[6, 11, … 25]",6,11,14,20,25,"""06""","""11""","""14""","""20""","""25""","""08""","""35""",true,16,"""05_03_06_05""",-9
"""04/02/2025""","""4117""","""34 35 06 23 13""","""0613233435""","[6, 13, … 35]",6,13,23,34,35,"""06""","""13""","""23""","""34""","""35""","""14""","""34""",false,0,"""07_10_11_01""",-9
"""30/01/2025""","""4112""","""23 06 22 33 34""","""0622233334""","[6, 22, … 34]",6,22,23,33,34,"""06""","""22""","""23""","""33""","""34""","""04""","""32""",true,16,"""16_01_10_01""",-42
"""25/01/2025""","""4107""","""06 07 25 29 31""","""0607252931""","[6, 7, … 31]",6,7,25,29,31,"""06""","""07""","""25""","""29""","""31""","""03""","""35""",false,0,"""01_18_04_02""",43
"""22/01/2025""","""4104""","""06 12 35 17 18""","""0612171835""","[6, 12, … 35]",6,12,17,18,35,"""06""","""12""","""17""","""18""","""35""","""01""","""30""",false,0,"""06_05_01_17""",23


df_jumps_map FOR str_b1 = 06


jumps_map,count
str,u32
"""02_08_06_07""",2
"""01_03_01_17""",2
"""03_04_12_10""",2
"""12_10_04_01""",2
"""13_06_02_06""",2
…,…
"""05_02_06_16""",1
"""02_15_10_02""",1
"""01_11_09_08""",1


df_unijump FOR str_b1 = 06


unijump,count
i64,u32
-9,15
3,15
5,15
0,15
7,14
…,…
-48,1
56,1
-36,1


In [7]:
possible_blast, exclude_jump_maps_blast = plot_post_col(col='str_b5', post_col='post_str_blast', test_value="27")

df_viz FOR str_b5 = 27


post_str_blast,count,percent
str,u32,f64
"""35""",55,14.986376
"""32""",50,13.623978
"""34""",36,9.809264
"""33""",34,9.264305
"""31""",34,9.264305
…,…,…
"""17""",2,0.544959
"""21""",2,0.544959
"""16""",2,0.544959


:Overlay
   .Bars.I    :Bars   [post_str_blast]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b5 = 27


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""15/03/2025""","""4156""","""16 15 05 27 02""","""0205151627""","[2, 5, … 27]",2,5,15,16,27,"""02""","""05""","""15""","""16""","""27""","""10""","""30""",true,16,"""03_10_01_11""",35
"""11/01/2025""","""4093""","""22 02 25 27 16""","""0216222527""","[2, 16, … 27]",2,16,22,25,27,"""02""","""16""","""22""","""25""","""27""","""04""","""23""",true,16,"""14_06_03_02""",-3
"""08/01/2025""","""4090""","""02 27 08 17 24""","""0208172427""","[2, 8, … 27]",2,8,17,24,27,"""02""","""08""","""17""","""24""","""27""","""05""","""32""",true,16,"""06_09_07_03""",3
"""20/12/2024""","""4071""","""08 22 23 07 27""","""0708222327""","[7, 8, … 27]",7,8,22,23,27,"""07""","""08""","""22""","""23""","""27""","""05""","""34""",true,16,"""01_14_01_04""",42
"""06/12/2024""","""4057""","""09 18 27 10 05""","""0509101827""","[5, 9, … 27]",5,9,10,18,27,"""05""","""09""","""10""","""18""","""27""","""04""","""25""",true,16,"""04_01_08_09""",-16
"""29/10/2024""","""4019""","""15 14 27 21 06""","""0614152127""","[6, 14, … 27]",6,14,15,21,27,"""06""","""14""","""15""","""21""","""27""","""05""","""31""",true,24,"""08_01_06_06""",-17
"""14/10/2024""","""4004""","""11 27 20 23 05""","""0511202327""","[5, 11, … 27]",5,11,20,23,27,"""05""","""11""","""20""","""23""","""27""","""01""","""29""",false,0,"""06_09_03_04""",16
"""25/08/2024""","""3954""","""27 10 21 05 23""","""0510212327""","[5, 10, … 27]",5,10,21,23,27,"""05""","""10""","""21""","""23""","""27""","""06""","""35""",false,0,"""05_11_02_04""",26
"""10/08/2024""","""3939""","""27 03 19 08 04""","""0304081927""","[3, 4, … 27]",3,4,8,19,27,"""03""","""04""","""08""","""19""","""27""","""10""","""33""",false,0,"""01_04_11_08""",-14


df_jumps_map FOR str_b5 = 27


jumps_map,count
str,u32
"""07_02_06_11""",2
"""09_04_02_01""",2
"""03_14_02_01""",2
"""02_04_08_02""",2
"""09_03_02_03""",2
…,…
"""01_14_06_02""",1
"""02_01_13_05""",1
"""05_01_08_05""",1


df_unijump FOR str_b5 = 27


unijump,count
i64,u32
-3,12
-10,11
-12,9
-4,9
-7,9
…,…
44,1
-46,1
-34,1


In [8]:
b_count = rr.CHOOSE
min_b = rr.MIN_B_VALUES[0]
max_b = rr.MAX_B_VALUES[-1]


combos = combinations(range(min_b, max_b + 1), b_count)
# print("generating combos: ", len(list(combos)))

In [9]:
lst = []
post_str_b1 = None
post_str_blast = None
for row in combos:
  cols = []
  str_row = " ".join([str(r).zfill(2) for r in row])
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=str_row)
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_blast = str_combo[-1]

with open('./gn_cahe_v3.json', 'w') as file: 
  json.dump(lst, file)

df_Orows = pl.DataFrame(
  lst,
  schema=[
    # 'fecha', 'sorteo', 'bolillas',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
    'post_str_b1', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 5),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(df_Orows)
print(f'{TOTAL_LINES=}')
display(df_Orows.head(10))

TOTAL_LINES=324632


id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0102030405""","[1, 2, … 5]",1,2,3,4,5,"""01""","""02""","""03""","""04""","""05""",null,null,true,8,"""01_01_01_01""",0
"""0102030406""","[1, 2, … 6]",1,2,3,4,6,"""01""","""02""","""03""","""04""","""06""","""01""","""05""",true,8,"""01_01_01_02""",1
"""0102030407""","[1, 2, … 7]",1,2,3,4,7,"""01""","""02""","""03""","""04""","""07""","""01""","""06""",true,8,"""01_01_01_03""",2
"""0102030408""","[1, 2, … 8]",1,2,3,4,8,"""01""","""02""","""03""","""04""","""08""","""01""","""07""",true,8,"""01_01_01_04""",3
"""0102030409""","[1, 2, … 9]",1,2,3,4,9,"""01""","""02""","""03""","""04""","""09""","""01""","""08""",true,8,"""01_01_01_05""",4
"""0102030410""","[1, 2, … 10]",1,2,3,4,10,"""01""","""02""","""03""","""04""","""10""","""01""","""09""",true,8,"""01_01_01_06""",5
"""0102030411""","[1, 2, … 11]",1,2,3,4,11,"""01""","""02""","""03""","""04""","""11""","""01""","""10""",true,8,"""01_01_01_07""",6
"""0102030412""","[1, 2, … 12]",1,2,3,4,12,"""01""","""02""","""03""","""04""","""12""","""01""","""11""",true,8,"""01_01_01_08""",7
"""0102030413""","[1, 2, … 13]",1,2,3,4,13,"""01""","""02""","""03""","""04""","""13""","""01""","""12""",true,8,"""01_01_01_09""",8


In [10]:
df_Orows.filter(is_rare_combination=False)

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0102040513""","[1, 2, … 13]",1,2,4,5,13,"""01""","""02""","""04""","""05""","""13""","""01""","""12""",false,0,"""01_02_01_08""",10
"""0102040523""","[1, 2, … 23]",1,2,4,5,23,"""01""","""02""","""04""","""05""","""23""","""01""","""22""",false,0,"""01_02_01_18""",20
"""0102040530""","[1, 2, … 30]",1,2,4,5,30,"""01""","""02""","""04""","""05""","""30""","""01""","""29""",false,0,"""01_02_01_25""",27
"""0102040531""","[1, 2, … 31]",1,2,4,5,31,"""01""","""02""","""04""","""05""","""31""","""01""","""30""",false,0,"""01_02_01_26""",28
"""0102040532""","[1, 2, … 32]",1,2,4,5,32,"""01""","""02""","""04""","""05""","""32""","""01""","""31""",false,0,"""01_02_01_27""",29
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2829313435""","[28, 29, … 35]",28,29,31,34,35,"""28""","""29""","""31""","""34""","""35""","""28""","""35""",false,0,"""01_02_03_01""",-3
"""2830313334""","[28, 30, … 34]",28,30,31,33,34,"""28""","""30""","""31""","""33""","""34""","""28""","""35""",false,0,"""02_01_02_01""",-4
"""2830313435""","[28, 30, … 35]",28,30,31,34,35,"""28""","""30""","""31""","""34""","""35""","""28""","""35""",false,0,"""02_01_03_01""",-7


In [13]:
# possible_b1, exclude_jump_maps_b1

df_possible = df_Orows.filter(
  # pl.col("str_b1").is_in(possible_b1),
  pl.col("str_b1").is_in(['11', '12', '13', '14', '15', '16', '17', '18']),
  # ~pl.col("str_b1").is_in(['18', '19', '20', '21', '22', '23', '24', '26']),
  ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_blast),
  pl.col("str_b5").is_in(possible_blast),
  ~pl.col("str_b5").is_in(['30', '23', '32']),
  pl.col("is_rare_combination") == False
)

df_possible

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""1112141531""","[11, 12, … 31]",11,12,14,15,31,"""11""","""12""","""14""","""15""","""31""","""11""","""30""",false,0,"""01_02_01_16""",18
"""1112141533""","[11, 12, … 33]",11,12,14,15,33,"""11""","""12""","""14""","""15""","""33""","""11""","""32""",false,0,"""01_02_01_18""",20
"""1112141534""","[11, 12, … 34]",11,12,14,15,34,"""11""","""12""","""14""","""15""","""34""","""11""","""33""",false,0,"""01_02_01_19""",21
"""1112141535""","[11, 12, … 35]",11,12,14,15,35,"""11""","""12""","""14""","""15""","""35""","""11""","""34""",false,0,"""01_02_01_20""",22
"""1112141731""","[11, 12, … 31]",11,12,14,17,31,"""11""","""12""","""14""","""17""","""31""","""11""","""30""",false,0,"""01_02_03_14""",10
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1829313435""","[18, 29, … 35]",18,29,31,34,35,"""18""","""29""","""31""","""34""","""35""","""18""","""35""",false,0,"""11_02_03_01""",-13
"""1829323335""","[18, 29, … 35]",18,29,32,33,35,"""18""","""29""","""32""","""33""","""35""","""18""","""34""",false,0,"""11_03_01_02""",-3
"""1829323435""","[18, 29, … 35]",18,29,32,34,35,"""18""","""29""","""32""","""34""","""35""","""18""","""35""",false,0,"""11_03_02_01""",-7


In [18]:
df_possible.sample(n=1)

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""1319213234""","[13, 19, … 34]",13,19,21,32,34,"""13""","""19""","""21""","""32""","""34""","""13""","""33""",false,0,"""06_02_11_02""",-31
